In [23]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],axis=1)

In [25]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


#### Attemp1: Dropping two non-beneficial columns 'STATUS' and 'SPECIAL_CONSIDERATIONS' to improve accuracy

In [26]:

application_df = application_df.drop(['STATUS','SPECIAL_CONSIDERATIONS'],axis=1)

In [27]:
# Determine the number of unique values in each column.
#print(application_df.apply(lambda col: col.unique()))
print(application_df.nunique())

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [28]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE']. value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [29]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [30]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [31]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]
#application_df.loc[application_df['CLASSIFICATION'].value_counts()>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [32]:
filtered_application_df = application_df.groupby('CLASSIFICATION').filter(lambda x : len(x)>1)

In [33]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C2100','C7000','C1700','C4000','C5000','C1270','C2700','C2800','C7100','C1300','C1280','C1230','C1400','C7200','C2300','C1240','C8000','C7120','C1500','C1800','C6000','C1250','C8200','C1238','C1278','C1235','C1237','C7210','C2400','C1720','C4100','C1257','C1600','C1260','C2710','C0','C3200','C1234','C1246','C1267','C1256']


# Replace in dataframe
for cls in classifications_to_replace:
    filtered_application_df['CLASSIFICATION'] = filtered_application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
filtered_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     4118
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [34]:
# Convert categorical data to numeric with `pd.get_dummies`
df_application_dummies = pd.get_dummies(filtered_application_df)
print(df_application_dummies.columns)

Index(['ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C3000', 'CLASSIFICATION_Other',
       'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999'

In [35]:
# Split our preprocessed data into our features and target arrays
y = df_application_dummies.IS_SUCCESSFUL.values
X = df_application_dummies.drop(columns = "IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=39))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3200      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,661
Trainable params: 5,661
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5708 - accuracy: 0.7214
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5572 - accuracy: 0.7263
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5521 - accuracy: 0.7321
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7335
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7328
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5721 - accuracy: 0.7219 - 423ms/epoch - 2ms/step
Loss: 0.5721405148506165, Accuracy: 0.7219045162200928


### Attemp2: Adding more neurons to a hidden layer
####    1.Increased no of nurons in first hidden layer
####    2.reducing the number of epochs to the training regimen.

In [73]:
### # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=320, activation="relu", input_dim=39))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 320)               12800     
                                                                 
 dense_28 (Dense)            (None, 30)                9630      
                                                                 
 dense_29 (Dense)            (None, 1)                 31        
                                                                 
Total params: 22,461
Trainable params: 22,461
Non-trainable params: 0
_________________________________________________________________


In [74]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [75]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5688 - accuracy: 0.7227
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5559 - accuracy: 0.7288
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7313
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7315
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7332
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7327
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7335
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7335
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7346
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7339

In [76]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5657 - accuracy: 0.7246 - 257ms/epoch - 959us/step
Loss: 0.5656766891479492, Accuracy: 0.7245886325836182


### Attempt3 : Adding more hidden layers

In [236]:
### # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=45, activation="relu", input_dim=39))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

#Third hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_170 (Dense)           (None, 45)                1800      
                                                                 
 dense_171 (Dense)           (None, 15)                690       
                                                                 
 dense_172 (Dense)           (None, 15)                240       
                                                                 
 dense_173 (Dense)           (None, 1)                 16        
                                                                 
Total params: 2,746
Trainable params: 2,746
Non-trainable params: 0
_________________________________________________________________


In [237]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [238]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=75)

Epoch 1/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5759 - accuracy: 0.7198
Epoch 2/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5568 - accuracy: 0.7302
Epoch 3/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7326
Epoch 4/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7325
Epoch 5/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5509 - accuracy: 0.7316
Epoch 6/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7329
Epoch 7/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7344
Epoch 8/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7342
Epoch 9/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7338
Epoch 10/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7334

In [239]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5666 - accuracy: 0.7226 - 381ms/epoch - 1ms/step
Loss: 0.5665891170501709, Accuracy: 0.7226047515869141
